## Mini Project #5 - Object Detection using SIFT

## Coded by Lucas Gomes 

First, create a program to register a template image from the camera stram

In [6]:
import cv2
import numpy as np

frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0) #capture a video from your default webcam 0, you can use 1 or 2 if you have more cameras
cap.set(3, frameWidth)#set the image width
cap.set(4, frameHeight)#set the image height
cap.set(10,30) # set the brightness defined as the id number 10
while True:
    success, img = cap.read() #read an image from the video and put it in img – success is a boolean
    if success:
        cv2.imshow("Result", img) #show the image
        if cv2.waitKey(1) & 0xFF == ord('q'): #if we go out a certain time or if q is pressed break the loop
            break
            cv2.destroyAllWindows()
        #Time to break the loop
    else:
        break
        cv2.destroyAllWindows()
cv2.destroyAllWindows()
image_template=img
cv2.imwrite(r"C:\Users\lucas\OneDrive\Documentos\computer_vision\Practice Part Four\images\templateVideo.png", image_template)
cv2.imshow("template",image_template)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()


Now, you will write a program to detect the template using the number of SIFT matches

In [7]:
import cv2
import numpy as np

frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0) #capture a video from your default webcam 0, you can use 1 or 2 if you have more cameras
cap.set(3, frameWidth)#set the image width
cap.set(4, frameHeight)#set the image height
cap.set(10,30) # set the brightness defined as the id number 10
while True:
    success, img = cap.read() #read an image from the video and put it in img – success is a boolean
    if success:
        cv2.imshow("Result", img) #show the image
        if cv2.waitKey(1) & 0xFF == ord('q'): #if we go out a certain time or if q is pressed break the loop
            break
            cv2.destroyAllWindows()
        #Time to break the loop
    else:
        break
        cv2.destroyAllWindows()
cv2.destroyAllWindows()
#write a function 
def sift_detector(new_image, image_template):
    # Function that compares input image to template
    # It then returns the number of SIFT matches between them
    
    #BGR2GRAY of the 2 images  
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    #image2 = cv2.cvtColor(image_template, cv2.COLOR_BGR2GRAY)
    image2 = image_template.copy()
    # Create SIFT detector object
    sift = cv2.SIFT_create()

    # Obtain the keypoints and descriptors using SIFT
    keypoints_1, descriptors_1 = sift.detectAndCompute(image1, None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(image2, None)

    # Define parameters for our Flann Matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 3)
    search_params = dict(checks = 100)

    # Create the Flann Matcher object
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Obtain matches using K-Nearest Neighbor Method
    # the result 'matchs' is the number of similar matches found in both images
    matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)

    # Store good matches using Lowe's ratio test
    good_matches = []
    for m,n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m) 

    return len(good_matches)

# Load our image template, this is our reference image
image_template = cv2.imread(r"C:\Users\lucas\OneDrive\Documentos\computer_vision\Practice Part Four\images\templateVideo.png", 0)  

# Get height and width of webcam frame
height, width = image_template.shape[:2]

# Define ROI Box Dimensions
top_left_x = width / 3
top_left_y = (height / 2) + (height / 4)
bottom_right_x = (width / 3) * 2
bottom_right_y = (height / 2) - (height / 4)
    
top_left_x = int(top_left_x)
top_left_y = int(top_left_y)
bottom_right_x = int(bottom_right_x)
bottom_right_y = int(bottom_right_y)

# Crop window of observation we defined above
croppedTemp = image_template[bottom_right_y:top_left_y , top_left_x:bottom_right_x]
cv2.imshow("Cropped Image Template", croppedTemp) 
cv2.waitKey(0)

cap = cv2.VideoCapture(0)

while True:
    #cv2.imshow("Cropped Image Template", croppedTemp) 
    # Get webcam images
    ret, frame = cap.read()

    # Get height and width of webcam frame
    height, width = frame.shape[:2]
    # Define ROI Box Dimensions
    top_left_x = width / 3
    top_left_y = (height / 2) + (height / 4)
    bottom_right_x = (width / 3) * 2
    bottom_right_y = (height / 2) - (height / 4)
    
    top_left_x = int(top_left_x)
    top_left_y = int(top_left_y)
    bottom_right_x = int(bottom_right_x)
    bottom_right_y = int(bottom_right_y)
    # Draw rectangular window for our region of interest   
    cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,0,255), 3)
    
    # Crop window of observation we defined above
    cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]
    cv2.imshow("Cropped Image ", cropped)
 
    # Flip frame orientation horizontally
    #frame = cv2.flip(frame,1)
    
    # Get number of SIFT matches
    matches = sift_detector(cropped, croppedTemp)

    # Display status string showing the current no. of matches 
    cv2.putText(frame,str(matches),(450,450), cv2.FONT_HERSHEY_COMPLEX, 2,(0,255,0),1)
    
    # Our threshold to indicate object deteciton
    # We use 10 since the SIFT detector returns little false positves
    threshold = 20
    
    # If matches exceed our threshold then object has been detected
    if matches > threshold:
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
        cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
    
    cv2.imshow('Object Detector using SIFT', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()   

#### Flannbased matching is quite fast, but not the most accurate. Other matching methods include:

- BruteForce
- BruteForce-SL2 (not in the documentation, BUT this is the one that skeeps the squared root !)
- BruteForce-L1
- BruteForce-Hamming
- BruteForce-Hamming(2)
